# 1.4.2 Non-MicroPython Device Version

This notebook serves as a replacement of [hardware/software communication notebook](./1.4-hardware-software-communication) if you were running on a device that supports Python (e.g., Raspberry Pi 5, Linux, Windows, MacOS, etc.) where you can install the `paho-mqtt` Python library (as opposed to using the `mqtt_as` MicroPython library on a Pico W). This notebook can also be used directly with [the companion notebook](./1.4.1-onboard-led-temp.ipynb). Note that these examples do not consider the possibility of having multiple devices or sending simultaneous requests, for simplicity. However, in [`self-driving-lab-demo`](https://self-driving-lab-demo.readthedocs.io/en/latest/), this is distinguished by passing an `experiment_id` back and forth, which is incorporated into the module assignment.

In [5]:
# only install paho-mqtt if we are running in colab
# matplotlib only for visualization
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    %pip install paho-mqtt

In [10]:
import paho.mqtt.client as mqtt
import ssl
import time
import random
import json

# Replace with your actual HiveMQ Cloud credentials and device ID
# (or import from your own my_secrets.py)
HIVEMQ_HOST = "248cc294c37642359297f75b7b023374.s2.eu.hivemq.cloud"
HIVEMQ_USERNAME = "sgbaird"
HIVEMQ_PASSWORD = "D.Pq5gYtejYbU#L"
COURSE_ID = "<your_id_here>"  # Make sure this matches your Colab script!

command_topic = f"{COURSE_ID}/onboard_led"
sensor_data_topic = f"{COURSE_ID}/onboard_temp"

# Simulate LED on/off
led_state = False # we will set this as a global var later, to emulate a hardware state


def read_temperature():
    # for demonstration
    return round(25 + random.uniform(-2.0, 2.0), 1)

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc, properties=None):
    if rc == 0:
        # Subscribing in on_connect() means that if we lose the connection and
        # reconnect then subscriptions will be renewed.
        client.subscribe(command_topic, qos=1)
    else:
        print(f"Failed to connect. Return code={rc}")

def on_message(client, userdata, msg):
    # we permit a global var for sake of demo'ing pretend hardware
    # normally the "state" would be kept by the device itself (hence one would remove the following line)
    global led_state
    try:
        payload_str = msg.payload.decode().strip().lower()
        print(f"Received command on {msg.topic}: {payload_str}")

        if msg.topic == command_topic:
            if msg == "on":
                led_state = True
            elif msg == "off":
                led_state = False
            elif msg == "toggle":
                led_state = not led_state
            else:
                print(f"Unknown command: {payload_str}")
            print(f"LED state: {led_state}")
            temperature = read_temperature()
            payload = f"{temperature}"
            client.publish(sensor_data_topic, payload, qos=1)
    except Exception as exc:
        print("Exception in on_message:", exc)

print("Setting up MQTT Client")
client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2, protocol=mqtt.MQTTv5)

# enable TLS for secure connection
client.tls_set(tls_version=mqtt.ssl.PROTOCOL_TLS_CLIENT)
# set username and password
client.username_pw_set(HIVEMQ_USERNAME, HIVEMQ_PASSWORD)
# connect to HiveMQ Cloud on port 8883
port = 8883
client.connect(HIVEMQ_HOST, port)
print("Connected to MQTT Client")
client.subscribe(sensor_data_topic, qos=2)

# Attach callbacks
client.on_connect = on_connect
client.on_message = on_message

# Start the MQTT network loop in a separate thread
client.loop_start()
print("Waiting for commands...")

start_time = time.time()
try:
    # Keep the script running to handle incoming messages
    while True:
        elapsed = round(time.time() - start_time)
        print(f"Running... Elapsed: {elapsed}s")
        time.sleep(5)
except KeyboardInterrupt:
    print("Interrupted by user.")
finally:
    # Gracefully stop
    client.loop_stop()
    client.disconnect()


Setting up MQTT Client
Connected to MQTT Client
Waiting for commands...
Running... Elapsed: 0s
Received command on <your_id_here>/onboard_led: abc
Unknown command: abc
LED state: False
Received command on <your_id_here>/onboard_temp: 24.9
Received command on <your_id_here>/onboard_led: abc
Unknown command: abc
LED state: False
Received command on <your_id_here>/onboard_temp: 24.4
Received command on <your_id_here>/onboard_led: abc
Unknown command: abc
LED state: False
Running... Elapsed: 5s
Received command on <your_id_here>/onboard_temp: 26.7
Received command on <your_id_here>/onboard_led: abc
Unknown command: abc
LED state: False
Received command on <your_id_here>/onboard_temp: 26.7
Received command on <your_id_here>/onboard_led: abc
Unknown command: abc
LED state: False
Received command on <your_id_here>/onboard_temp: 26.4
Interrupted by user.
